In [1]:
# system imports
import sys
import os
import datetime
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# import pickle
import dill
# from copy import deepcopy

# pyomo imports
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import beautify, beautify_reactive, HiddenLogs, HiddenPrints, plot_distribution
from utility.model_utility import add_dual, update_dual, delete_dual, check_DOF, check_iteration, \
                            augmented_objective, add_solver, select_MPCC, which_MPCC
# from utility.data_utility import cal_cnumber
from utility.time_utility import create_filename_time, log_now, log_end

2018-07-23 23:02:16 - Start Program


In [2]:
model = pe.ConcreteModel(name='reactive_distillation')

In [3]:
'''
model input
'''
tray_number = 20
non_reactive_flag = [1,2,3,4,5,6,7,13,19,20]
rr_ratio = 0.05
side_draw_flag = {4:0.04,7:0.2,10:0.3}
# default temperature is 220C
temperature_flag = {}
temperature_flag.update({j:200 + 10*(j-8) for j in [8,9,10,11,12]})
temperature_flag.update({j:260 + 10*(j-14) for j in [14,15,16]})
temperature_flag.update({j:280 + 10*(j-16) for j in [17,18]})

'''
construct the reactive stages
'''
model.TRAY = pe.RangeSet(1,tray_number)
model.TRAY_nonreactive = pe.Set(initialize=non_reactive_flag)
model.TRAY_reactive = model.TRAY - model.TRAY_nonreactive

# with HiddenPrints():
#     model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)
#     add_dual(pe,model)
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)
add_dual(pe,model)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].H_F
| reactive[1].T
| reactive[1].T_F
| reactive[1].P
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].r_total_comp
| reactive[1].PR_L
| reactive[1].PR_V
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_b

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[4].VLE_block.P_VLE
| reactive[4].VLE_block.n_ave
| reactive[4].VLE_block.n_ave_cal
| reactive[4].VLE_block.Hen
| reactive[4].VLE_block.Hen0
| reactive[4].VLE_block.gamma
| reactive[4].VLE_block.P_sat
| reactive[4].VLE_block.P_sat_Y
| reactive[4].VLE_block.P_sat_dY_inf
| reactive[4].VLE_block.P_sat_dY0
| reactive[4].VLE_block.Hen_ref
| reactive[4].VLE_block.Hen0_ref
| reactive[4].VLE_block.gamma_ref
| reactive[4].VLE_block.V_L
| reactive[4].VLE_block.V_L_dY_inf
| reactive[4].VLE_block.V_L_dY0
| reactive[4].VLE_block.poynting
--------------------------------------------------

> Importing MPCC_P_pf Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[4].MPCC_P_pf.s_L
| reactive[4].MPCC_P_pf.s_V
| reactive[4].MPCC_P_pf.pf
| reactive[4].MPCC_P_pf.rho
--------------------------------------------------
>

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[8].VLE_block.P_VLE
| reactive[8].VLE_block.n_ave
| reactive[8].VLE_block.n_ave_cal
| reactive[8].VLE_block.Hen
| reactive[8].VLE_block.Hen0
| reactive[8].VLE_block.gamma
| reactive[8].VLE_block.P_sat
| reactive[8].VLE_block.P_sat_Y
| reactive[8].VLE_block.P_sat_dY_inf
| reactive[8].VLE_block.P_sat_dY0
| reactive[8].VLE_block.Hen_ref
| reactive[8].VLE_block.Hen0_ref
| reactive[8].VLE_block.gamma_ref
| reactive[8].VLE_block.V_L
| reactive[8].VLE_block.V_L_dY_inf
| reactive[8].VLE_block.V_L_dY0
| reactive[8].VLE_block.poynting
--------------------------------------------------

> Importing MPCC_P_pf Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[8].MPCC_P_pf.s_L
| reactive[8].MPCC_P_pf.s_V
| reactive[8].MPCC_P_pf.pf
| reactive[8].MPCC_P_pf.rho
--------------------------------------------------
>

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[12].VLE_block.P_VLE
| reactive[12].VLE_block.n_ave
| reactive[12].VLE_block.n_ave_cal
| reactive[12].VLE_block.Hen
| reactive[12].VLE_block.Hen0
| reactive[12].VLE_block.gamma
| reactive[12].VLE_block.P_sat
| reactive[12].VLE_block.P_sat_Y
| reactive[12].VLE_block.P_sat_dY_inf
| reactive[12].VLE_block.P_sat_dY0
| reactive[12].VLE_block.Hen_ref
| reactive[12].VLE_block.Hen0_ref
| reactive[12].VLE_block.gamma_ref
| reactive[12].VLE_block.V_L
| reactive[12].VLE_block.V_L_dY_inf
| reactive[12].VLE_block.V_L_dY0
| reactive[12].VLE_block.poynting
--------------------------------------------------

> Importing MPCC_P_pf Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[12].MPCC_P_pf.s_L
| reactive[12].MPCC_P_pf.s_V
| reactive[12].MPCC_P_pf.pf
| reactive[12].MPCC_P_pf.rho
-------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[16].VLE_block.P_VLE
| reactive[16].VLE_block.n_ave
| reactive[16].VLE_block.n_ave_cal
| reactive[16].VLE_block.Hen
| reactive[16].VLE_block.Hen0
| reactive[16].VLE_block.gamma
| reactive[16].VLE_block.P_sat
| reactive[16].VLE_block.P_sat_Y
| reactive[16].VLE_block.P_sat_dY_inf
| reactive[16].VLE_block.P_sat_dY0
| reactive[16].VLE_block.Hen_ref
| reactive[16].VLE_block.Hen0_ref
| reactive[16].VLE_block.gamma_ref
| reactive[16].VLE_block.V_L
| reactive[16].VLE_block.V_L_dY_inf
| reactive[16].VLE_block.V_L_dY0
| reactive[16].VLE_block.poynting
--------------------------------------------------

> Importing MPCC_P_pf Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[16].MPCC_P_pf.s_L
| reactive[16].MPCC_P_pf.s_V
| reactive[16].MPCC_P_pf.pf
| reactive[16].MPCC_P_pf.rho
-------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[20].VLE_block.P_VLE
| reactive[20].VLE_block.n_ave
| reactive[20].VLE_block.n_ave_cal
| reactive[20].VLE_block.Hen
| reactive[20].VLE_block.Hen0
| reactive[20].VLE_block.gamma
| reactive[20].VLE_block.P_sat
| reactive[20].VLE_block.P_sat_Y
| reactive[20].VLE_block.P_sat_dY_inf
| reactive[20].VLE_block.P_sat_dY0
| reactive[20].VLE_block.Hen_ref
| reactive[20].VLE_block.Hen0_ref
| reactive[20].VLE_block.gamma_ref
| reactive[20].VLE_block.V_L
| reactive[20].VLE_block.V_L_dY_inf
| reactive[20].VLE_block.V_L_dY0
| reactive[20].VLE_block.poynting
--------------------------------------------------

> Importing MPCC_P_pf Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[20].MPCC_P_pf.s_L
| reactive[20].MPCC_P_pf.s_V
| reactive[20].MPCC_P_pf.pf
| reactive[20].MPCC_P_pf.rho
-------------------------------